In [2]:
import numpy as np
#if this fails, you need to put the case_studies.py file in the same folder
from case_studies import *

In [3]:
from scipy.optimize import minimize

#These are the example optimizers you should evaluate this week.
#These ar eoptimizers implemented in scipy.
#they take as first 2 or 3 arguments the function f, its gradient df and sometimes its hessian Hf.
#the next parameters are all the same: x0 is the starting point, max_iterations the stopping criterion for iterations
#and epsilon the precision tolerance to be reached. 
#Note: epsilon is interpreted slightly differently across algorithms, and some algorithms might not reach the tolerance
#and stop early.
def scipy_bfgs(f,df,x0,max_iterations,epsilon):
    xs=[]
    grad_norms=[]
    def logging_f(x):
        xs.append(x)
        grad_norms.append(np.maximum(np.linalg.norm(df(x)),10**(-5)*epsilon))
        return f(x)
    minimize(logging_f, x0, method="BFGS", jac=df, tol=epsilon,options={'maxiter':max_iterations, 'gtol':epsilon})
    return np.array(xs), np.array(grad_norms)

def scipy_newton(f,df,Hf,x0,max_iterations,epsilon):
    xs=[]
    grad_norms=[]
    def logging_f(x):
        xs.append(x)
        grad_norms.append(np.maximum(np.linalg.norm(df(x)),10**(-5)*epsilon))
        return f(x)
    minimize(logging_f, x0, method="Newton-CG", jac=df, hess=Hf, tol=epsilon,options={'maxiter':max_iterations,'xtol':epsilon})
    return np.array(xs), np.array(grad_norms)

def scipy_trust_region(f,df,Hf,x0,max_iterations,epsilon):
    xs=[]
    grad_norms=[]
    def logging_f(x):
        xs.append(x)
        grad_norms.append(np.maximum(np.linalg.norm(df(x)),10**(-5)*epsilon))
        return f(x)
    minimize(logging_f, x0, method="trust-exact", jac=df, hess=Hf, tol=epsilon,options={'maxiter':max_iterations})
    return np.array(xs), np.array(grad_norms)

In [ ]:
#example usage of the algorithms
#the output is a list of points evaluated on the function as well as the gradient norms at that point
#this algorithms has the first three arguments functions for function value, gradient and Hessian.
#For the 5 functions, those are named f1-f5 etc and cna be found in the case_studies.py file
x0=np.ones(2)
xs,grad_norms = scipy_trust_region(f4,df4,Hf4,x0, 1000, 1.e-10)


#the optimal point for a given function and dimensionality is stored in the package as well for at least 15 decimals precision
optimal = x_opt("f4", 2)
print("final solution point:", xs[-1])
print("distance of x from optimum", np.linalg.norm(xs[-1]-optimal))
print("number of function evaluations:", len(grad_norms))
print("final function value:", f4(xs[-1]))
print("final gradient norm:", grad_norms[-1])


final solution point: [0.00190933 0.00190933]
distance of x from optimum 1.1857524696459338e-11
number of function evaluations: 9
final function value: 1.2203194789920598e-05
final gradient norm: 9.977761920841575e-11
Name: f4


In [7]:
#Evaluation function
def evaluate_optimizer(f, df, optimizer, x0, max_iterations, eps, Hf=None):
    if Hf is not None:
        xs, grad_norms = optimizer(f, df, Hf, x0, max_iterations, eps)
    else:
        xs, grad_norms = optimizer(f, df, x0, max_iterations, eps)
    
    optimizer_name = optimizer.__name__
    fname = f.__name__

    optimal_solution = x_opt(fname, len(x0))
    final_solution_point = xs[-1]
    dist_to_optimum = np.linalg.norm(xs[-1]-optimal_solution)
    num_of_fun_evals = len(grad_norms)
    final_fun_value = f(xs[-1])
    final_grad_norm = grad_norms[-1]
    
    print(f"Function: {fname}, Optimizer: {optimizer_name}")
    print( 
    f"""Final Solution Point: {final_solution_point}
    Distance of x to optimum: {dist_to_optimum}
    Number of Function Evaluations: {num_of_fun_evals}
    Final Function Value: {final_fun_value}
    Final Gradient Norm: {final_grad_norm}
    """)

    return {
        "optimizer": optimizer_name,
        "function": fname,
        "distance_to_optimum": dist_to_optimum,
        "num_evaluations": num_of_fun_evals,
        "final_function_value": final_fun_value,
        "final_gradient_norm": final_grad_norm,
    }



In [9]:
#Ellipsoid function evaluation
x0 = np.ones(2)
max_iterations = 1000
eps = 1.e-10

f1_bfgs = evaluate_optimizer(f1, df1, scipy_bfgs, x0, max_iterations, eps)
f1_newton = evaluate_optimizer(f1, df1, scipy_newton, x0, max_iterations, eps, Hf1)
f1_trust_region= evaluate_optimizer(f1, df1, scipy_trust_region, x0, max_iterations, eps, Hf1)

Function: f1, Optimizer: scipy_bfgs
Final Solution Point: [1.08420217e-19 0.00000000e+00]
    Distance of x to optimum: 1.0842021724855044e-19
    Number of Function Evaluations: 6
    Final Function Value: 1.1754943508222875e-38
    Final Gradient Norm: 1e-15
    
Function: f1, Optimizer: scipy_newton
Final Solution Point: [0. 0.]
    Distance of x to optimum: 0.0
    Number of Function Evaluations: 3
    Final Function Value: 0.0
    Final Gradient Norm: 1e-15
    
Function: f1, Optimizer: scipy_trust_region
Final Solution Point: [ 2.22044605e-16 -1.73472348e-18]
    Distance of x to optimum: 2.220513810852149e-16
    Number of Function Evaluations: 3
    Final Function Value: 5.231307211441829e-32
    Final Gradient Norm: 3.497753190081524e-15
    
